In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [24]:
def make_data():
    imageData=[]
    labels=[]
    data=[]
    features=28*28*3
    noImage=96000
    nClass=96
    path=os.getcwd()
    folderpath=os.path.join(path,"classes")

    label=-1
    for length in range(2):
        for width in range(2):
            for angle in range(12):
                for color in range(2):

                    string=str(length)+"_"+str(width)+"_"+str(angle)+"_"+str(color)
                    fpath=folderpath+"/"+string

                    ipath=fpath+"/"+str(length)+"_"+str(width)+"_"+str(angle)+"_"+str(color)
                    label=label+1
                    
                    for image in range(1000):
                        iName=ipath+"_"+str(image)+".jpeg"
                        img_data=cv2.imread(iName)
                        data.append(img_data)#.reshape(features))
                        labels.append(label)
    
    numbers = np.arange(96000)
    print(numbers[-1])
    imageData=np.array(data,float)#.reshape(noImage,features)
    labels=np.array(labels,float).reshape(noImage,1)
    print("data shape: ",imageData.shape)
    print("labels shape: ",labels.shape)
    
    imageData=imageData/255.0
    data=np.concatenate((imageData,labels),axis=1)
    np.random.shuffle(data)
    data=data.reshape(noImage,features+1)
    
    ipt=data[:,:-1]
    opt=np.array(data[:,-1],int).reshape(noImage,1)
    b=np.zeros((noImage,nClass),int)
    for i in range(noImage):
        b[i,opt[i]]=1
        
    return ipt,opt

In [23]:
def getImageVariable(path):
    classFolder=path
#     imagePath="/classes"
    imageList=[]
    cnt=0
    tempArry=np.zeros((96000,5),int)

    for length in range(2):
        for width in range(2):
            for angle in range(12):
                for color in range(2):
                    
                    for variation in range(10):
                        tempArry[cnt,:]=[cnt,length,width,angle,color]
                        cnt+=1
    np.random.shuffle(tempArry)
    
    for i in range(96000):
        string="/"+str(tempArry[i][1])+"_"+str(tempArry[i][2])+"_"+str(tempArry[i][3])+"_"+str(tempArry[i][4])
        imgFolder=classFolder+string
        img=imgFolder+string+"_"+str(tempArry[i][0]%1000)+".jpeg"
        imageList.append(cv2.imread(img))
        
    return imageList,tempArry[:,1],tempArry[:,2],tempArry[:,3],tempArry[:,4]


imagePath="./classes" # jaha imagees ka folder h
a,b,c,d,e=getImageVariable(imagePath)
print(a[0],b[0],c[0],d[0],e[0])

[[[ 7  2  0]
  [ 4  1  0]
  [ 2  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 4  0  0]
  [ 2  0  1]
  [ 0  0  8]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  9]
  [ 0  0 30]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 ...

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]] 0 0 0 0


In [17]:
x,y=make_data()

data shape:  (96000, 28, 28, 3)
labels shape:  (96000, 1)


ValueError: all the input arrays must have same number of dimensions

In [4]:
nx = x
ny = y

print(nx.shape)
print(ny.shape)

(96000, 2352)
(96000, 1)


In [5]:
flat_list = []
for sublist in ny:
    for item in sublist:
        flat_list.append(item)
zy = flat_list
# print(y.shape)
print(zy[np.argmax(zy)])

95


In [6]:
splitSize = int(nx.shape[0]*0.6)
nx, testData = nx[:splitSize,:], nx[splitSize:,:]
zy, testY = zy[:splitSize], zy[splitSize:]
print(zy[:20])

[85, 50, 36, 42, 34, 56, 39, 42, 28, 86, 48, 28, 0, 61, 55, 82, 70, 38, 75, 71]


In [7]:
print(nx.shape)
# print(zy.shape)
nx = nx.astype('float32')
# nx /= 255
nx = nx.reshape(nx.shape[0], 28, 28, 3)
testData = testData.reshape(testData.shape[0], 28, 28, 3)
print(nx.shape)
zy = np.array(zy)
testY = np.array(testY)
print(zy.shape)
print(testData.shape)
print(testY.shape)

# testData = testData.astype('float32')
# testData /= 255
# testData = testData.reshape(x.shape[0], 28, 28, 1)

(67200, 2352)
(67200, 28, 28, 3)
(67200,)
(28800, 28, 28, 3)
(28800,)


In [8]:
# print(y[:10])
# y_lab = []
# cnt = 0
# for val in y:
#     if cnt < 10:
#         print(val)
#         cnt += 1
#     y_lab.append(np.argmax(val)+1)
# y = np.array(y_lab)

# print(y[:10])

# y_lab = []
# for val in testY:
#     y_lab.append(np.argmax(val)+1)
# testY = y_lab

In [11]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Conv2D(36, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model2.add(tf.keras.layers.BatchNormalization())
# model2.add(tf.keras.layers.Conv2D(36, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
# model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model2.add(tf.keras.layers.Conv2D(36, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
model2.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(1024, activation="relu"))

model2.add(tf.keras.layers.Dense(96, activation='softmax'))

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.summary()

# print(y_train.shape)

model2.fit(nx, zy, batch_size=64, epochs=3)

score = model2.evaluate(testData, testY)
# pred = model2.predict(testData)
print('\n', 'Test Acc = ', score[1])
# print('\n'," Predictions: ", pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 36)        5328      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 36)        0         
_________________________________________________________________
batch_normalization_v1_2 (Ba (None, 14, 14, 36)        144       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 36)        63540     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
batch_normalization_v1_3 (Ba (None, 7, 7, 36)          144       
_________________________________________________________________
flatten_2 (Flatten)          (None, 1764)              0         
__________

In [13]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (7, 7), strides=1, activation='relu', padding="same", input_shape=(28, 28,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation="relu"))

model.add(tf.keras.layers.Dense(96, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


# print(y_train.shape)

model.fit(nx, zy, batch_size=64, epochs=3)

score = model.evaluate(testData, testY)
# pred = model2.predict(testData)
print('\n', 'Test Acc = ', score[1])
# print('\n'," Predictions: ", pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 32)        4736      
_________________________________________________________________
batch_normalization_v1_5 (Ba (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              6423552   
_________________________________________________________________
dense_9 (Dense)              (None, 96)                98400     
Total params: 6,526,816
Trainable params: 6,526,752
Non-trainable params: 64
_________________________________________________________________